In [ ]:
%matplotlib inline
from lewidi_lib import load_preds, process_rdf
import pandas as pd

reasoning_df = pd.read_json("../rate_reasoning.jsonl", lines=True)
reasoning_df

In [2]:
from lewidi_lib import assign_cols_perf_metrics, join_correct_responses


joint_df = (
    load_preds("../parquets")
    .merge(
        reasoning_df,
        on=["model_id", "dataset", "split", "dataset_idx", "template_id", "run_idx"],
    )
    .pipe(process_rdf, discard_invalid_pred=True)
    .pipe(join_correct_responses)
    .pipe(assign_cols_perf_metrics)
)

In [3]:
def rate_reasoning(sent_anns: list[dict]) -> float:
    oks = [ann["annotation"] == "1 - ok" for ann in sent_anns]
    return sum(oks) / len(oks)


joint_df = joint_df.assign(
    reasoning_score=joint_df["sentence_annotations"].apply(rate_reasoning)
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.regplot(
    data=joint_df[["reasoning_score", "ws_loss"]],
    x="reasoning_score",
    y="ws_loss",
)
ax.grid(alpha=0.5)